In [1]:
from vedo import Mesh, Box, Sphere, ConvexHull
import numpy as np
from normalize import get_eigenvectors, normalize_shape, get_center_of_mass

In [ ]:
boxMesh = normalize_shape(Box(width=1,height=1,length=1).c("Black").wireframe(True))
sphereMesh = normalize_shape(Sphere(r=1, res=500, quads=False, c='red', alpha=1.0))
train = normalize_shape(Mesh("..\shapes\Train\D01014.obj"))
head = normalize_shape(Mesh("..\shapes\HumanHead\D00131.obj"))
insect = normalize_shape(Mesh("..\shapes\Insect\D00117.obj"))

In [7]:
def extract_features(mesh:Mesh):
    cvx = ConvexHull(mesh.vertices)
    diameterRet = get_diameter(mesh,cvx)
    mesh.fill_holes()
    mesh.triangulate()
    ret = {
        "area":mesh.area(),
        "volume":mesh.volume(),
        "rectangularity":get_rectangularity(mesh),
        "compactness":get_compactness(mesh),
        "convexity": get_convexity(mesh,cvx),
        "eccentricity":get_eccentricity(mesh),
        "diameter":diameterRet[0],
        "diameterPts":diameterRet[1],
        "distributions":get_distributions(mesh)
    }
    return ret

def get_surface_area(mesh:Mesh):
    #area = sqrt(fabs(s * (s - a) * (s - b) * (s - c)));
    return mesh.area()

def get_rectangularity(mesh:Mesh):
    #How close is the shape (post normalisation to its oriented bounding box)
    # (shape volume divided by OBB volume)
    bbox = mesh.bounds()
    Dx = np.abs(bbox[0] - bbox[1])  
    Dy = np.abs(bbox[2] - bbox[3])  
    Dz = np.abs(bbox[4] - bbox[5])  
    obbVol = Dx*Dy*Dz
    rectangularity = mesh.volume()/obbVol
    return rectangularity

def get_compactness(mesh:Mesh):
    #How close is the shape to a sphere
    return mesh.area()**3/(36*np.pi*(mesh.volume()**2))
    #return mesh.area()**1.5/(36*np.pi*(mesh.volume()**0.5))
    #return mesh.area()**3/(mesh.volume()**2)

def get_convexity(mesh:Mesh,cvx:ConvexHull):
    #(shape volume divided by convex hull volume)
    convexity = mesh.volume()/cvx.volume()
    return convexity


def get_diameter(mesh:Mesh,cvx:ConvexHull,k=500):
    maxD = 0
    maxP = [None,None]
    if(len(cvx.vertices)<k):
        subs=cvx.vertices
    else:
        subs = cvx.vertices[np.random.choice(cvx.vertices.shape[0], k, replace=False)]
    for v1 in subs:
        for v2 in cvx.vertices:
            d = np.linalg.norm(v1-v2)
            if d>maxD:
                maxD=d
                maxP = [v1,v2]
    return maxD,maxP        
        
        

def get_eccentricity(mesh:Mesh):
    #ratio of largest to smallest eigenvalues of covariance matrix
    _,eigval = get_eigenvectors(mesh)
    mineig = min(eigval)
    maxeig = max(eigval)
    return np.abs(maxeig)/np.abs(mineig)


def get_distributions(mesh:Mesh, show=True):
    com = get_center_of_mass(mesh)
    subsample1= mesh.vertices[np.random.choice(mesh.vertices.shape[0], 5, replace=False), :]
 
    D1 = calc_D1(com,subsample1)

    distributions = {
        "D1":D1
    }
    return distributions


#Calculate distance between the center and a random subset, returns list of euclidian distances
def calc_D1(center, subs):
    ret = []
    for pt in subs:
        ret.append(np.linalg.norm(pt - center))
    return ret
    
print("Box:",extract_features(boxMesh))
print("Sphere:",extract_features(sphereMesh))
print("Train:",extract_features(train))
print("Head:",extract_features(head))
print("Insect:",extract_features(insect))
    

Box: {'area': 5.999999999999999, 'volume': 0.9999999999999998, 'rectangularity': 0.9999999999999998, 'compactness': 1.9098593171027443, 'convexity': 1.0, 'eccentricity': 1.0, 'diameter': 1.7320508, 'diameterPts': [array([ 0.5, -0.5, -0.5], dtype=float32), array([-0.5,  0.5,  0.5], dtype=float32)], 'distributions': {'D1': [0.7214622737884497, 0.6882098607982887, 0.6510208329999894, 0.6373774391990981, 0.5125381083392727]}}
Sphere: {'area': 3.1409621960160763, 'volume': 0.5233652114635619, 'rectangularity': 0.5234419936153972, 'compactness': 1.000290287391841, 'convexity': 0.9999998860995919, 'eccentricity': 1.006338105363994, 'diameter': 1.0001802, 'diameterPts': [array([ 0.17156927, -0.28756544,  0.3711504 ], dtype=float32), array([-0.17064126,  0.28963393, -0.37053242], dtype=float32)], 'distributions': {'D1': [0.49963202524441885, 0.5001050294325559, 0.5000972641446765, 0.4995267386181736, 0.4999862809171987]}}
Train: {'area': 1.642157399797742, 'volume': 0.0013122085996907806, 'rect

Difficult Calculations
Area - Positive vs Negative (RIght hand rule, counterclockwise orientation = positive, thumb away from screen)
    Solution 1) Ignore sign -> Problem Concave Shapes have "negative" areas -> Areas are overstimated
    Same applies for volume
    Solution 2) Incooperate Sign -> Allow negative areas, concavities are incooperated. Problem: consistent orientation along boundary
    FOr VOlumes check orientation of triangles. -> Get Consistently oriented triangles.

HOle Filling:
Every Edge is shared by two triangles. If an edge has a single edge -> it is a boundary. If you have a boundary loop -> fill. 


In [25]:
print(len(head.vertices))
chull = ConvexHull(head.vertices)
print(len(chull.vertices))

5161
1809
